# **ETL datos de YELP**
---


In [3]:
# Importacion de librerias a utilizar
from google.colab import drive
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json
import pickle
import pyarrow.parquet as pq

# Establece conexión con google Drive
drive.mount('/content/drive')

def aplicar_ETL():
  # Variables donde se almacenan las rutas de los archivos crudos
  ruta1='/content/drive/MyDrive/Proyecto Final/Yelp/business.pkl'
  ruta2='/content/drive/MyDrive/Proyecto Final/Yelp/checkin.json'
  ruta3='/content/drive/MyDrive/Proyecto Final/Yelp/review.json'
  # ruta4='/content/drive/MyDrive/Proyecto Final/Yelp/tip.json'
  ruta5='/content/drive/MyDrive/Proyecto Final/Yelp/user.parquet'

  rutas=[ruta1,ruta2,ruta3,ruta5]

  for ruta in rutas:
    nombre_archivo=ruta.split('/')[-1].split('.')[0]


    # Desarrollo de ETL---------------------------------
    if nombre_archivo=='business':
      try:
        #  # Lee archivo y devuelve un dataframe de pandas
        business=pd.read_pickle(ruta)
        business=business.iloc[:,:14]
        # Seleccion de columna importantes
        df_business=business.drop(columns=['is_open','attributes','hours'])
        # Elimina valores nulos
        df_business=df_business.dropna().reset_index(drop=True)
        # Cambio de Tipos
        df_business['latitude']=df_business['latitude'].astype('float')
        df_business['longitude']=df_business['longitude'].astype('float')
        df_business['stars']=df_business['stars'].astype('int')
        df_business['review_count']=df_business['review_count'].astype('int')
        # Lista de estados seleccionados
        estados_seleccionados=['PA','FL','CA','IL']
        # Filtado registros por estados
        df_business=df_business[df_business['state'].isin(estados_seleccionados)].reset_index(drop=True)
        # Trabajaremos con los negocios que contengan mas de 20 reseñas, ya que menos reseñas no seria eficiente para el sistema de recomendación
        df_business=df_business[df_business['review_count']>20].reset_index(drop=True)
        # Filtrado de negocios por rubro gastronómico
        palabras_clave=["Restaurants","Food","Bubble Tea","Coffee & Tea","Bakery","Barbecue","Buffets","Burgers","Cafes","Catering","Cocktail Bars","Desserts","Diners","Food Trucks","Ice Cream & Frozen Yogurt","Juice Bars & Smoothies","Pizza","Seafood","Steakhouses","Sushi Bars","Tapas Bars","Wine Bars","Vegetarian","Vegan","Gluten-Free"]
        filtro_gastronomico=df_business['categories'].str.contains('|'.join(palabras_clave),case=False)
        df_business=df_business[filtro_gastronomico].reset_index(drop=True)
        # Exportar archivo en formato parquet
        # df_business.to_parquet('business_procesado.parquet')
      except: pass


    elif nombre_archivo=='checkin':
      try:
        # Lee archivo y devuelve un dataframe de pandas
        checkin=pd.read_json(ruta,lines=True,nrows=200000)
        # Transforma los datos de la columna "date" en una lista de fechas
        checkin['date']=checkin['date'].str.split(',')
        # Expandir datos
        checkin=checkin.explode('date').reset_index(drop=True)
        # Cruzar con datos de df_business y filtrar unicamente los negocios del rubro gastronomico
        df_merge_checkin_business=pd.merge(checkin,df_business,on='business_id',how='inner')
        df_checkin=df_merge_checkin_business[checkin.columns]
        # Exportar archivo en formato parquet
        df_checkin.to_parquet('checkin_procesado.parquet')
      except: pass


    elif nombre_archivo=='review':
      try:
        # Lee archivo y devuelve un dataframe de pandas
        review=pd.read_json(ruta,lines=True,nrows=200000)
        # Cambiar nombre de variable "stars" para evitar errores al momento de cruzar los datos
        review.rename({'stars':'stars_review'},axis=1,inplace=True)
        # Cruzar con datos del data set con df_biusiness (resultado luego del etl del dataset "business")
        df_review=pd.merge(review,df_business,on='business_id',how='inner')
        df_review=df_review[review.columns]
        # Exportar archivo en formato parquet
        df_review.to_parquet('review_procesado.parquet')
      except: pass


    # elif nombre_archivo=='user':
    #   try:
    #     tabla_parquet=pq.read_table(ruta5)
    #     user=tabla_parquet.to_pandas()
    #     # Lee archivo y devuelve un dataframe de pandas
    #     # user=pd.read_parquet(ruta5)
    #     #Eliminar registros duplicados
    #     df_user=user.drop_duplicates().reset_index(drop=True)
    #     # Filtrar usuarios con mayor cantidad de reseñas efectuadas (usuarios mas activos en la plataforma es un indicador de fiabilidad del comentario tanto positivo como negativo)
    #     df_user=df_user[df_user['review_count']>20]
    #     # Exportar archivo en formato parquet
    #     df_user.to_parquet('user_procesado.parquet')
    #   except:pass


  # # Muestra datasets ya procesados
  print('\n---------------------------------------------------------------------Dataset "business" procesado-------------------------------------------------------\n')
  display(df_business)
  print('\n---------------------------------------------------------------------Dataset "checkin" procesado--------------------------------------------------------\n')
  display(checkin)
  print('\n---------------------------------------------------------------------Dataset "review" procesado----------------------------------------------------------\n')
  display(review)
  # print('\n---------------------------------------------------------------------Dataset "user" procesado----------------------------------------------------------\n')
  # display(df_user)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
aplicar_ETL()


---------------------------------------------------------------------Dataset "business" procesado-------------------------------------------------------



,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,categories
0,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,CA,19107,39.955505,-75.155564,4,80,"Restaurants, Food, Bubble Tea, Coffee & Tea, B..."
1,0bPLkL0QhhPO5kt1_EXmNQ,Zio's Italian Market,2575 E Bay Dr,Largo,FL,33771,27.916116,-82.760461,4,100,"Food, Delis, Italian, Bakeries, Restaurants"
2,QdN72BWoyFypdGJhhI5r7g,Bar One,767 S 9th St,Philadelphia,PA,19147,39.939825,-75.157447,4,65,"Cocktail Bars, Bars, Italian, Nightlife, Resta..."
3,Mjboz24M9NlBeiOJKLEd_Q,DeSandro on Main,4105 Main St,Philadelphia,PA,19127,40.022466,-75.218314,3,41,"Pizza, Restaurants, Salad, Soup"
4,aPNXGTDkf-4bjhyMBQxqpQ,Craft Hall,901 N Delaware Ave,Philadelphia,PA,19123,39.962582,-75.135657,3,65,"Eatertainment, Arts & Entertainment, Brewpubs,..."
...,...,...,...,...,...,...,...,...,...,...,...
17021,wVxXRFf10zTTAs11nr4xeA,PrimoHoagies,6024 Ridge Ave,Philadelphia,CA,19128,40.032483,-75.214430,3,55,"Restaurants, Specialty Food, Food, Sandwiches,..."
17022,8n93L-ilMAsvwUatarykSg,Kitchen Gia,3716 Spruce St,Philadelphia,PA,19104,39.951018,-75.198240,3,22,"Coffee & Tea, Food, Sandwiches, American (Trad..."
17023,2MAQeAqmD8enCT2ZYqUgIQ,The Melting Pot - Nashville,"166 2nd Ave N, Ste A",Nashville,PA,37201,36.163875,-86.776311,4,204,"Fondue, Beer, Wine & Spirits, Food, Restaurants"
17024,w_4xUt-1AyY2ZwKtnjW0Xg,Bittercreek Alehouse,246 N 8th St,Boise,PA,83702,43.616590,-116.202383,4,998,"Bars, Gastropubs, Sandwiches, Nightlife, Resta..."



---------------------------------------------------------------------Dataset "checkin" procesado--------------------------------------------------------



,business_id,date
0,---kPU91CF4Lq2-WlRu9Lw,2020-03-13 21:10:56
1,---kPU91CF4Lq2-WlRu9Lw,2020-06-02 22:18:06
2,---kPU91CF4Lq2-WlRu9Lw,2020-07-24 22:42:27
3,---kPU91CF4Lq2-WlRu9Lw,2020-10-24 21:36:13
4,---kPU91CF4Lq2-WlRu9Lw,2020-12-09 21:23:33
...,...,...
13356870,zzu6_r3DxBJuXcjnOYVdTw,2013-12-11 00:52:49
13356871,zzu6_r3DxBJuXcjnOYVdTw,2013-12-13 00:58:14
13356872,zzw66H6hVjXQEt0Js3Mo4A,2016-12-03 23:33:26
13356873,zzw66H6hVjXQEt0Js3Mo4A,2018-12-02 19:08:45



---------------------------------------------------------------------Dataset "review" procesado----------------------------------------------------------



,review_id,user_id,business_id,stars_review,useful,funny,cool,text,date
0,KU_O5udG6zpxOg-VcAEodg,mh_-eMZ6K5RLWhZyISBhwA,XQfwVwDr-v0ZS3_CbbE5Xw,3,0,0,0,"If you decide to eat here, just be aware it is...",2018-07-07 22:09:11
1,BiTunyQ73aT9WBnpR9DZGw,OyoGAe7OKpv6SyGZT5g77Q,7ATYjTIgM3jUlt4UM3IypQ,5,1,0,1,I've taken a lot of spin classes over the year...,2012-01-03 15:28:18
2,saUsX_uimxRlCVr67Z4Jig,8g_iMtfSiwikVnbP2etR0A,YjUWPpI6HXG530lwP-fb2A,3,0,0,0,Family diner. Had the buffet. Eclectic assortm...,2014-02-05 20:30:30
3,AqPFMleE6RsU23_auESxiA,_7bHUi9Uuf5__HHc_Q8guQ,kxX2SOes4o-D3ZQBkiMRfA,5,1,0,1,"Wow! Yummy, different, delicious. Our favo...",2015-01-04 00:01:03
4,Sx8TMOWLNuJBWer-0pcmoA,bcjbaE6dDog4jkNY91ncLQ,e4Vwtrqf-wpJfwesgvdgxQ,4,1,0,1,Cute interior and owner (?) gave us tour of up...,2017-01-14 20:54:15
...,...,...,...,...,...,...,...,...,...
199995,nRYJWQ6YVjcrZCEU_JWrIQ,uakTslg3Ay3C0816-5yNdQ,idf-eiurCrbsLRcH7c9zmw,5,0,0,0,Oakley's Bistro is a hidden gem for Indianapol...,2012-10-16 17:45:43
199996,4ylitTY7GmonnQDqhjB-eg,_fMw99otXOCu-Kzc-loMZA,v72HalBu5fQIFTD-oqhSgQ,5,0,0,0,"Let me just say, I'm glad my husband and I dec...",2016-08-18 00:23:31
199997,MnwNSr5h_t1uIlcj3aI_5w,ZfG1T_noQnbOdXnz4USs0Q,cvHY1RTAPSoH94ysP2_eyQ,5,1,1,1,I was coming back from the Farmers market toda...,2014-06-21 17:12:53
199998,3fp0uXGAjIqgza02_ObF5w,I0MMq4iYjDjERnSjp76HDw,cBp1fn2LXpSHe4VbTXLYEw,5,0,0,1,Absolutely perfect meal for me. The food is cl...,2017-09-06 17:54:23


In [ ]:
# ADAPTACION PARA GCP

# Importacion de librerias a utilizar
from google.colab import drive
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json
import pickle
import pyarrow.parquet as pq
import geopy.geocoders import Nominatim



# Desarrollo de ETL---------------------------------
if nombre_archivo=='business':
  try:
    #Filtrado de columnas
    datos=datos.iloc[:,:14]
    # Seleccion de columna importantes
    datos=datos.drop(columns=['is_open','attributes','hours'])
    # Elimina valores nulos
    datos=datos.dropna().reset_index(drop=True)
    # Cambio de Tipos
    datos['latitude']=datos['latitude'].astype('float')
    datos['longitude']=datos['longitude'].astype('float')
    datos['stars']=datos['stars'].astype('int')
    datos['review_count']=datos['review_count'].astype('int')
    # Lista de estados seleccionados
    estados_seleccionados=['PA','FL','CA','IL']
    # Filtado registros por estados
    datos=datos[datos['state'].isin(estados_seleccionados)].reset_index(drop=True)
    # Trabajaremos con los negocios que contengan mas de 20 reseñas, ya que menos reseñas no seria eficiente para el sistema de recomendación
    datos=datos[datos['review_count']>20].reset_index(drop=True)
    # Filtrado de negocios por rubro gastronómico
    palabras_clave=["Restaurants","Food","Bubble Tea","Coffee & Tea","Bakery","Barbecue","Buffets","Burgers","Cafes","Catering","Cocktail Bars","Desserts","Diners","Food Trucks","Ice Cream & Frozen Yogurt","Juice Bars & Smoothies","Pizza","Seafood","Steakhouses","Sushi Bars","Tapas Bars","Wine Bars","Vegetarian","Vegan","Gluten-Free"]
    filtro_gastronomico=datos['categories'].str.contains('|'.join(palabras_clave),case=False)
    datos=datos[filtro_gastronomico].reset_index(drop=True)
  except: pass


elif nombre_archivo=='checkin':
  try:
    # Transforma los datos de la columna "date" en una lista de fechas
    datos['date']=datos['date'].str.split(',')
    # Expandir datos
    datos=datos.explode('date').reset_index(drop=True)
  except: pass


elif nombre_archivo=='review':
  try:
    # Elimina valores nulos
    datos=datos.dropna().reset_index(drop=True)
    datos=datos.drop_duplicates().reset_index(drop=True)

  except: pass


elif nombre_archivo=='user':
  try:
    datos=datos.drop_duplicates().reset_index(drop=True)
    # Filtrar usuarios con mayor cantidad de reseñas efectuadas (usuarios mas activos en la plataforma es un indicador de fiabilidad del comentario tanto positivo como negativo)
    datos=datos[datos['review_count']>20]
  except:pass



### FILTRADO POR LONIGITUD Y LATITUD
---